# Transient Stokes Flow: A Parametric Study

**Objective:** This notebook performs a parametric study of a 2D transient Stokes flow simulation in an open channel. We will investigate the impact of mesh size and different implicit time-integration methods on solver performance.

**Methodology:**
1.  **FEM Setup:** Use `Ferrite.jl` to define the geometry, finite element spaces, and assemble the mass and stiffness matrices.
2.  **Time Integration:** Use `DifferentialEquations.jl` to solve the resulting system of differential-algebraic equations (DAEs).
3.  **Parametric Sweep:** Create a main simulation function and call it with different parameters for:
    - Mesh refinement level.
    - Implicit solver algorithm (e.g., `Rodas5P`, `ImplicitEuler`).
4.  **Profiling:** Analyze the `sol.stats` object to compare solver efficiency across different runs.

## 1. Package Imports and Setup

In [6]:
# Import required packages
using Ferrite
using SparseArrays
using LinearAlgebra
using BlockArrays
using DifferentialEquations
using Plots
using WriteVTK
using Printf

## 2. Core Finite Element Assembly Functions

These functions assemble the stiffness matrix `K` and the mass matrix `M` for the Stokes system. The system can be written as a DAE: `M u' + K u = 0`.

In [7]:
function assemble_stokes_matrix!(K, dh, cvu, cvp, viscosity)
    """
    Assemble the stiffness matrix for the Stokes system.
    K = [K_uu  K_up]
        [K_pu   0  ]
    """
    assembler = start_assemble(K)
    ke = zeros(ndofs_per_cell(dh), ndofs_per_cell(dh))
    
    # Get DOF ranges for velocity (:u) and pressure (:p)
    range_u = dof_range(dh, :u)
    ndofs_u = length(range_u)
    range_p = dof_range(dh, :p)
    ndofs_p = length(range_p)
    
    # Pre-allocate arrays for shape function values
    ∇ϕᵤ = Vector{Tensor{2,2,Float64,4}}(undef, ndofs_u)
    divϕᵤ = Vector{Float64}(undef, ndofs_u)
    ϕₚ = Vector{Float64}(undef, ndofs_p)
    
    # Element loop
    for cell in CellIterator(dh)
        Ferrite.reinit!(cvu, cell)
        Ferrite.reinit!(cvp, cell)
        ke .= 0
        
        # Quadrature loop
        for qp in 1:getnquadpoints(cvu)
            dΩ = getdetJdV(cvu, qp)
            
            for i in 1:ndofs_u
                ∇ϕᵤ[i] = shape_gradient(cvu, qp, i)
                divϕᵤ[i] = shape_divergence(cvu, qp, i)
            end
            for i in 1:ndofs_p
                ϕₚ[i] = shape_value(cvp, qp, i)
            end
            
            # K_uu: Viscous term ∫ μ ∇u : ∇v dΩ
            for (i, I) in pairs(range_u), (j, J) in pairs(range_u)
                ke[I, J] += viscosity * (∇ϕᵤ[i] ⊡ ∇ϕᵤ[j]) * dΩ
            end
            
            # K_up: Pressure gradient term ∫ -p ∇·v dΩ
            for (i, I) in pairs(range_u), (j, J) in pairs(range_p)
                ke[I, J] += (-divϕᵤ[i] * ϕₚ[j]) * dΩ
            end
            
            # K_pu: Continuity constraint ∫ -q ∇·u dΩ
            for (i, I) in pairs(range_p), (j, J) in pairs(range_u)
                ke[I, J] += (-divϕᵤ[j] * ϕₚ[i]) * dΩ
            end
        end
        assemble!(assembler, celldofs(cell), ke)
    end
    return K
end

function assemble_mass_matrix!(M, dh, cvu)
    """
    Assemble the mass matrix. Only the velocity block is non-zero.
    M = [M_uu  0]
        [0     0]
    """
    n_basefuncs_v = getnbasefunctions(cvu)
    
    # Local element mass matrix (only velocity block)
    Mₑ_uu = zeros(n_basefuncs_v, n_basefuncs_v)
    
    # We only need to assemble into the velocity-velocity block
    assembler = start_assemble(M)
    dof_range_u = dof_range(dh, :u)
    
    for cell in CellIterator(dh)
        fill!(Mₑ_uu, 0)
        Ferrite.reinit!(cvu, cell)
        
        for qp in 1:getnquadpoints(cvu)
            dΩ = getdetJdV(cvu, qp)
            for i in 1:n_basefuncs_v
                φᵢ = shape_value(cvu, qp, i)
                for j in 1:n_basefuncs_v
                    φⱼ = shape_value(cvu, qp, j)
                    Mₑ_uu[i, j] += (φᵢ ⋅ φⱼ) * dΩ
                end
            end
        end
        
        # Assemble the local velocity block into the global matrix
        cell_dofs = celldofs(cell)
        assemble!(assembler, cell_dofs[dof_range_u], Mₑ_uu)
    end
    return M
end

assemble_mass_matrix! (generic function with 1 method)

## 3. Main Simulation Function

This function encapsulates the entire simulation pipeline for a given set of parameters. This makes it easy to loop over different configurations.

In [8]:
# This struct and its method are crucial for DAEs with Dirichlet BCs.
# It tells the solver to only calculate the error norm on the unconstrained ("free")
# degrees of freedom, preventing the solver from getting confused by the
# externally-forced boundary condition values.
struct FreeDofErrorNorm
    ch::ConstraintHandler
end

# Method for when u is a single number (e.g., during initialization)
(fe_norm::FreeDofErrorNorm)(u::Number, t) = DiffEqBase.ODE_DEFAULT_NORM(u, t)

# Method for when u is a vector (during time stepping)
(fe_norm::FreeDofErrorNorm)(u::AbstractArray, t) = DiffEqBase.ODE_DEFAULT_NORM(u[fe_norm.ch.free_dofs], t)

function run_stokes_simulation(; nelem, solver, abstol, reltol, t_end, viscosity)
    
    println("\n--- Running Simulation ---")
    @printf("Mesh elements (y-dir): %d | Solver: %s | Tolerances: (a=%.0e, r=%.0e)\n", nelem, typeof(solver), abstol, reltol)

    # 1. MESH AND GEOMETRY
    H = 0.25
    L = 4 * H
    nels = (4 * nelem, nelem)
    grid = generate_grid(Quadrilateral, nels, Vec((0.0, 0.0)), Vec((L, H)))

    # 2. FINITE ELEMENT SPACES
    dim = 2
    degree = 1
    ipu = Lagrange{RefQuadrilateral, degree + 1}()^dim # Q2 for velocity
    ipp = Lagrange{RefQuadrilateral, degree}()      # Q1 for pressure
    dh = DofHandler(grid)
    add!(dh, :u, ipu)
    add!(dh, :p, ipp)
    close!(dh)

    qr = QuadratureRule{RefQuadrilateral}(2 * degree + 1)
    ipg = Lagrange{RefQuadrilateral, 1}()
    cvu = CellValues(qr, ipu, ipg)
    cvp = CellValues(qr, ipp, ipg)
    
    # 3. BOUNDARY CONDITIONS
    ch = ConstraintHandler(dh)
    vmax = 1.0
    vin(t) = min(t * vmax, vmax) # Ramped velocity from t=0 to t=1
    parabolic_inflow(x, t) = Vec((vin(t) * x[2] * (H - x[2]) / (H^2 / 4), 0.0))

    add!(ch, Dirichlet(:u, getfacetset(grid, "left"), parabolic_inflow))
    
    # *** FIX: The no-slip condition for a 2D vector field must return a 2D vector. ***
    add!(ch, Dirichlet(:u, union(getfacetset(grid, "top"), getfacetset(grid, "bottom")), (x, t) -> Vec((0.0, 0.0))))

    add!(ch, Dirichlet(:p, getfacetset(grid, "right"), (x, t) -> 0.0))
    close!(ch)

    # 4. SYSTEM ASSEMBLY
    # Use allocate_matrix for better performance with ConstraintHandler
    K = allocate_matrix(dh, ch)
    assemble_stokes_matrix!(K, dh, cvu, cvp, viscosity)
    
    M = allocate_matrix(dh, ch)
    assemble_mass_matrix!(M, dh, cvu)

    # 5. TIME INTEGRATION SETUP (DIFFERENTIALEQUATIONS.JL)
    # The DAE is M u' = -K u. We define the function f(u, p, t) = -K*u.
    function stokes_transient!(du, u, p, t)
        update!(ch, t)
        # *** FIX: Use apply! to correctly impose time-dependent BCs, not apply_zero! ***
        apply!(u, ch) 
        mul!(du, K, u)
        du .*= -1
    end

    # The Jacobian of f(u) is simply -K. Apply BCs for the solver.
    jac_sparsity = copy(K) # Use the assembled K as the sparsity pattern
    K_applied = copy(K)
    apply!(K_applied, ch) # This modifies K_applied in-place
    jac_func = (J, u, p, t) -> (J .= -K_applied)

    # Define the DAE function with mass matrix and analytical Jacobian
    f = ODEFunction(stokes_transient!, mass_matrix=M, jac=jac_func, jac_prototype=jac_sparsity)
    
    # Set initial conditions (u=0 at t=0)
    u0 = zeros(ndofs(dh))
    update!(ch, 0.0)
    apply!(u0, ch) # Ensure initial conditions satisfy BCs at t=0

    # Define and solve the problem
    prob = ODEProblem(f, u0, (0.0, t_end))
    sol = solve(prob, solver, abstol=abstol, reltol=reltol, progress=true, progress_steps=1, internalnorm=FreeDofErrorNorm(ch))
    
    return sol, dh
end

run_stokes_simulation (generic function with 1 method)

## 4. Post-Processing and Visualization

This function takes a solution and visualizes the velocity and pressure fields.

In [9]:
function postprocess_and_plot(u_final, dh, nels, title_suffix="")
    # This function extracts field values at element centers for plotting
    velx, vely, pres = Float64[], Float64[], Float64[]
    
    # Recreate CellValues for post-processing
    degree = 1
    qr = QuadratureRule{RefQuadrilateral}(1) # Evaluate at center
    ipu = Lagrange{RefQuadrilateral, degree + 1}()^2
    ipp = Lagrange{RefQuadrilateral, degree}()
    ipg = Lagrange{RefQuadrilateral, 1}()
    cvu = CellValues(qr, ipu, ipg)
    cvp = CellValues(qr, ipp, ipg)
    
    dof_range_u = dof_range(dh, :u)
    dof_range_p = dof_range(dh, :p)

    for cell in CellIterator(dh)
        Ferrite.reinit!(cvu, cell)
        Ferrite.reinit!(cvp, cell)
        
        u_local = u_final[celldofs(cell)[dof_range_u]]
        p_local = u_final[celldofs(cell)[dof_range_p]]
        
        # Get values at the first (and only) quadrature point
        u_vec = function_value(cvu, 1, u_local)
        p_val = function_value(cvp, 1, p_local)
        
        push!(velx, u_vec[1])
        push!(vely, u_vec[2])
        push!(pres, p_val)
    end

    # Reshape data into a grid for contour plotting
    velx2d = reshape(velx, nels)
    vely2d = reshape(vely, nels)
    pres2d = reshape(pres, nels)

    # Create plots
    p1 = contourf(velx2d', title="x-velocity (u_x)", colorbar_title="u_x", aspect_ratio=:equal)
    p2 = contourf(vely2d', title="y-velocity (u_y)", colorbar_title="u_y", aspect_ratio=:equal)
    p3 = contourf(pres2d', title="Pressure (p)", colorbar_title="p", aspect_ratio=:equal)

    plot_title = "Transient Stokes Solution at t_end" * title_suffix
    combined_plot = plot(p1, p2, p3, layout=(3,1), size=(800, 900), plot_title=plot_title)
    
    display(combined_plot)
end

postprocess_and_plot (generic function with 2 methods)

## 5. Running the Parametric Study

Now we define the parameter sets and loop through them, running the simulation for each case and printing the solver statistics.

In [10]:
# --- Simulation Parameters ---
const T_END = 2.0         # Final simulation time
const VISCOSITY = 1e-2    # Lower viscosity for more interesting transient behavior
const ABSTOL = 1e-5
const RELTOL = 1e-5

# --- Parameter Sets to Test ---
mesh_sizes = [10, 20] # Number of elements in y-direction
solvers = [
    Rodas5P(autodiff=false),        # A higher-order stiffly-accurate solver
    KenCarp4(autodiff=false)        # Another popular stiff solver that can handle DAEs
]

# --- Main Loop ---
for nelem in mesh_sizes
    for solver_method in solvers
        
        # Use @time to measure the total execution time
        elapsed_time = @elapsed begin
            sol, dh = run_stokes_simulation(
                nelem=nelem,
                solver=solver_method,
                abstol=ABSTOL,
                reltol=RELTOL,
                t_end=T_END,
                viscosity=VISCOSITY
            )
        end
        
        # --- Print Profiling Information ---
        println("--- Profiling Results ---")
        @printf("Total elapsed time: %.2f seconds\n", elapsed_time)
        println("Solver stats:")
        stats = sol.stats
        @printf("  Accepted steps: %d\n", stats.naccept)
        @printf("  Rejected steps: %d\n", stats.nreject)
        @printf("  RHS evaluations: %d\n", stats.nf)
        @printf("  Jacobian evaluations: %d\n", stats.njacs)
        @printf("  Linear factorizations (nw): %d\n", stats.nw)
        println("-------------------------\n")
        
        # --- Optional: Visualize the final state for the last run ---
        if nelem == last(mesh_sizes) && solver_method == last(solvers)
            println("Generating final plot...")
            nels = (4 * nelem, nelem)
            postprocess_and_plot(sol.u[end], dh, nels, " (nelem=$nelem, solver=$(typeof(solver_method)))")
            
            # --- Optional: Save a time series to VTK for ParaView ---
            println("Saving VTK time series...")
            pvd = paraview_collection("stokes-transient-series")
            for (i, t) in enumerate(sol.t)
                vtk_grid("stokes-transient-series-$i", dh) do vtk
                    write_solution(vtk, dh, sol.u[i])
                    pvd[t] = vtk
                end
            end
            vtk_save(pvd)
            println("VTK series saved to stokes-transient-series.pvd")
        end
    end
end



--- Running Simulation ---
Mesh elements (y-dir): 10 | Solver: Rodas5P{0, false, Nothing, typeof(OrdinaryDiffEqCore.DEFAULT_PRECS), Val{:forward}, true, nothing, typeof(OrdinaryDiffEqCore.trivial_limiter!), typeof(OrdinaryDiffEqCore.trivial_limiter!)} | Tolerances: (a=1e-05, r=1e-05)
--- Profiling Results ---


┌ Warning: At t=0.0, dt was forced below floating point epsilon 5.0e-324, and step error estimate = NaN. Aborting. There is either an error in your model specification or the true solution is unstable (or the true solution can not be represented in the precision of Float64).
└ @ SciMLBase C:\Users\Maan\.julia\packages\SciMLBase\tey0W\src\integrator_interface.jl:620


Total elapsed time: 71.78 seconds
Solver stats:
  Accepted steps: 0
  Rejected steps: 548
  RHS evaluations: 4436
  Jacobian evaluations: 25
  Linear factorizations (nw): 548
-------------------------


--- Running Simulation ---
Mesh elements (y-dir): 10 | Solver: KenCarp4{0, false, Nothing, NLNewton{Rational{Int64}, Rational{Int64}, Rational{Int64}, Rational{Int64}}, typeof(OrdinaryDiffEqCore.DEFAULT_PRECS), Val{:forward}, true, nothing, typeof(OrdinaryDiffEqCore.trivial_limiter!)} | Tolerances: (a=1e-05, r=1e-05)


LoadError: This solver is not able to use mass matrices. For compatible solvers see https://docs.sciml.ai/DiffEqDocs/stable/solvers/dae_solve/

## 6. Analysis and Next Steps

After running the simulations above, you should analyze the printed statistics. Here is what to look for:

1.  **Mesh Refinement (`nelem`):**
    - **Effect on Time:** How much does doubling the elements in each direction (a 4x increase in total DOFs) increase the total run time?
    - **Effect on Steps:** Does a finer mesh require more or fewer time steps? Often, it requires more because smaller elements can introduce higher-frequency dynamics that the solver needs to resolve, or the Courant–Friedrichs–Lewy (CFL) condition becomes more restrictive.

2.  **Solver Choice (`solver`):**
    - **`ImplicitEuler`:** This is the simplest method. It is very stable but only first-order accurate. It will likely take many small steps to meet the tolerance, but each step is computationally cheaper.
    - **`Rosenbrock23` and `Rodas5P`:** These are higher-order methods designed for stiff problems like this one. They can take much larger time steps than `ImplicitEuler` while still meeting the error tolerance. Compare `naccept` between them. `Rodas5P` (5th order) should take fewer steps than `Rosenbrock23` (2nd/3rd order), but each step is more expensive. The overall winner depends on which trade-off is better for this specific problem.
    - **Overall Time:** Which solver gives the lowest total elapsed time for a given mesh and tolerance?

3.  **Solver Statistics (`sol.stats`):**
    - **`nreject`:** A high number of rejected steps indicates the solver is struggling. This might happen if the problem is very stiff or if the tolerances are too tight for the chosen method.
    - **`nf` vs `njacs`:** For Rosenbrock methods, the number of function evaluations (`nf`), Jacobian evaluations (`njacs`), and linear solves (`nw`) are closely related. Comparing these can give insight into the cost per step of each algorithm.

**Your Next Task:** Based on these initial findings, you can conduct a more detailed study. For example, you could:
- Fix the mesh and solver and vary the `abstol` and `reltol` to see how they affect performance.
- Choose the most efficient solver and run it on a very fine mesh to get a high-fidelity result.
- Investigate the effect of changing the `viscosity`.